In [1]:
import torch
from utils import *
from dataset import ImageDataset
from models.swin_unet import SwinUNet
from dataset import ImageDataset
import os
from glob import glob

from tqdm import tqdm
from utils import *
from consts import *
import cv2
import numpy as np
from datetime import datetime
from PIL.Image import fromarray

In [2]:
checkpoint = torch.load("/cluster/scratch/csicard/checkpoints/swin-unet/best_val_patch_acc_0.919625_epoch_24.pt", map_location=torch.device('cpu'))
model = SwinUNet(model_type="base").to("cpu")
model.load_state_dict(checkpoint["model_state_dict"])

<All keys matched successfully>

In [9]:
device = "cpu"
test_path = "../data/test"


print("Predicting on test set...")
test_path = os.path.join(test_path, "images")
test_filenames = glob(test_path + "/*.png")
test_images = load_all_from_path(test_path)
batch_size = test_images.shape[0]
size = test_images.shape[1:3]
# we also need to resize the test images. This might not be the best ideas depending on their spatial resolution.
print("Resizing test images...")
test_images = np.stack([img for img in test_images], 0)
test_images = test_images[:, :, :, :3]


Predicting on test set...


Loading images: 100%|██████████| 144/144 [00:00<00:00, 162.66it/s]


Resizing test images...


In [19]:
%matplotlib inline

print("Making predictions...")
import torchvision.transforms.functional as TF


# checkpoint = torch.load(best_weights_path)
# model.load_state_dict(checkpoint["model_state_dict"])
# print(f"Loaded best model weights ({best_weights_path})")

test_pred = []
CROP_SIZE = 200
RESIZE_SIZE = 208
for image in tqdm(test_images):
    # 1. On fait les crops: on divise l'image en 4 parties qui ne s'overlapent pas
    cropped_image = [
        image[:, 0:CROP_SIZE, 0:CROP_SIZE],
        image[:, CROP_SIZE : 2 * CROP_SIZE, 0:CROP_SIZE],
        image[:, 0:CROP_SIZE, CROP_SIZE : 2 * CROP_SIZE],
        image[:, CROP_SIZE : 2 * CROP_SIZE, CROP_SIZE : 2 * CROP_SIZE],
    ]
    pred_cropped = np.zeros((len(cropped_image), 3, RESIZE_SIZE, RESIZE_SIZE))

    for i, c_img in enumerate(cropped_image):
        r = cv2.resize(c_img, dsize=(RESIZE_SIZE, RESIZE_SIZE))
        print("Shape resize", r.shape)
        print("Max resize", np.max(r))
        t = np_to_tensor(np.expand_dims(np.moveaxis(r, -1, 0), axis=0), device)
        print("To tensor + unsqueeze shape", t.shape)
        print("To tensor + unsqueeze max", t.max())
        res = model(t).detach().cpu().numpy()
        print("After model shape:", res.shape)
        print("After model max:", np.max(res))
        raise Exception()
        print(res.shape)
        
        pred_cropped[i, :, :, :] = res


    print(f"Max of all pred cropeed images: {torch.max(pred_cropped)}")

    full_pred = np.zeros((3, 400, 400))

    full_pred[:, 0:CROP_SIZE, 0:CROP_SIZE] = TF.resize(
        pred_cropped[0], size=(CROP_SIZE, CROP_SIZE), antialias=True
    )
    print(f"Max full pred: {np.max(full_pred)}")
    full_pred[:, CROP_SIZE : 2 * CROP_SIZE, 0:CROP_SIZE] = TF.resize(
        pred_cropped[1], size=(CROP_SIZE, CROP_SIZE), antialias=True
    )
    print(f"Max full pred: {np.max(full_pred)}")
    full_pred[:, 0:CROP_SIZE, CROP_SIZE : 2 * CROP_SIZE] = TF.resize(
        pred_cropped[2], size=(CROP_SIZE, CROP_SIZE), antialias=True
    )
    print(f"Max full pred: {np.max(full_pred)}")
    full_pred[
        :, CROP_SIZE : 2 * CROP_SIZE, CROP_SIZE : 2 * CROP_SIZE
    ] = TF.resize(pred_cropped[3], size=(CROP_SIZE, CROP_SIZE), antialias=True)
    print(f"Max full pred: {np.max(full_pred)}")
    test_pred.append(full_pred)

Making predictions...


  0%|          | 0/144 [00:00<?, ?it/s]

Shape resize (208, 208, 3)
Max resize 0.99607843
To tensor + unsqueeze shape torch.Size([1, 3, 208, 208])
To tensor + unsqueeze max tensor(0.9961)


  0%|          | 0/144 [00:00<?, ?it/s]

After model shape: (1, 1, 208, 208)
After model max: 0.1959082


Exception: 

In [ ]:

    print("max of test_pred", np.max(test_pred))
    # test_pred = [model(t).detach().cpu().numpy()
    #              for t in tqdm(test_images.unsqueeze(1))]
    test_pred = np.concatenate(test_pred, 0)
    test_pred = np.moveaxis(test_pred, 1, -1)  # CHW to HWC
    test_pred = np.stack([img for img in test_pred], 0)  # resize to original shape
    # Now compute labels
    test_pred = test_pred.reshape(
        (-1, size[0] // PATCH_SIZE, PATCH_SIZE, size[0] // PATCH_SIZE, PATCH_SIZE)
    )
    test_pred = np.moveaxis(test_pred, 2, 3)
    test_pred = np.round(np.mean(test_pred, (-1, -2)) > CUTOFF)
    print(f"Test predictions shape: {test_pred.shape}")
    now = datetime.now()
    t = now.strftime("%Y-%m-%d_%H-%M-%S")
    os.makedirs("submissions", exist_ok=True)
    create_submission(
        test_pred,
        test_filenames,
        submission_filename=f"../submissions/swin_unet_submission_{t}.csv",
    )
    print(f"Created submission!")

In [ ]:
test_pred_copy = test_pred.copy()

In [ ]:
np.max(test_pred[0])

In [ ]:

with torch.no_grad():
    test_pred_c = np.concatenate(test_pred_copy, 0)
    test_pred_c = np.moveaxis(test_pred, 1, -1)  # CHW to HWC
    test_pred_c = np.stack([img for img in test_pred], 0)  # resize to original shape
    # Now compute labels
    test_pred_c = test_pred_c.reshape(
        (-1, size[0] // PATCH_SIZE, PATCH_SIZE, size[0] // PATCH_SIZE, PATCH_SIZE)
    )
    test_pred_c = np.moveaxis(test_pred_c, 2, 3)
    test_pred_c = np.round(np.mean(test_pred_c, (-1, -2)) > CUTOFF)
    print(f"Test predictions shape: {test_pred_c.shape}")
    now = datetime.now()
    t = now.strftime("%Y-%m-%d_%H-%M-%S")
    os.makedirs("submissions", exist_ok=True)
    create_submission(
        test_pred_c,
        test_filenames,
        submission_filename=f"../submissions/swin_unet_submission_{t}.csv",
    )
    print(f"Created submission!")

In [11]:
a = torch.zeros((1,2, 3))
b = torch.zeros((1,2, 3))
c = torch.stack([a,b], 0)
c.shape

torch.Size([2, 1, 2, 3])

In [12]:
c[0, :, :, :].shape

torch.Size([1, 2, 3])

In [13]:
for i in c:
    print(i)

tensor([[[0., 0., 0.],
         [0., 0., 0.]]])
tensor([[[0., 0., 0.],
         [0., 0., 0.]]])
